**Frequency dressed with -40 dB:** 7.579121159 GHz

In [1]:
%matplotlib widget
import pyvisa
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.optimize import curve_fit


import sys 
sys.path.insert(0, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Actual_Work')
sys.path.insert(1, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Instruments\network\RS_Signal_Generator')
sys.path.insert(2, r'C:\Users\oper\SynologyDrive\Lab2023\KIDs\QTLab2324\PAandQubit\Instruments\network\VNA')

import FunFit as FF
import Tuls as Ts
import RS_SMA100B
import NA_N9916A

ip_sg           = '192.168.40.15' 
ip_vna          = '192.168.40.10' 
vna             = NA_N9916A.N9916A(ip = ip_vna)
sGen            = RS_SMA100B.SMA100B(ip = ip_sg)

N9916A:	Connection successful!
SMA100B:	Connection successful!


In [7]:
vna._resource.query('FREQ:DATA?')

# vna.read('FREQ:DATA?').decode('utf-8')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xbc in position 10: ordinal not in range(128)

In [2]:
f0 = vna.get_freqs()
f0 = np.mean(np.array([int(i) for i in f0.split(',')]))
# f0 = 7.579121159e9
# f0 = 7.5788290e9

mode        = 'NA'              
s_par       = 'S21'       
f_span      = 1
f_center    = f0
bw          = 10e3            # IF BANDWIDTH
n_avgs      = 1
n_pts       = 600 
power_lvl   = -40  
                            
###############################

vna.run_hold('RUN')
vna.set_num_pts(n_pts)
vna.set_freq_bandwidth(bw)
vna.set_freq_span(f_span)
vna.set_freq_center(f_center)
vna.num_avgs(n_avgs)
vna.power = power_lvl
vna.set_scaling(auto = True)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xbc in position 10: ordinal not in range(128)

In [5]:
f_sg_min    = 5.4e9
f_sg_max    = 5.8e9
f_step      = 10e6

lvl_sg_min  = -20
lvl_sg_max  = 2
lvl_step    = 1

freqs_sg    = np.arange(f_sg_min, f_sg_max, f_step)
lvl_sg      = np.arange(lvl_sg_min, lvl_sg_max, lvl_step)

print('len freqs:\t', len(freqs_sg))
print('len lvl:\t', len(lvl_sg))

len freqs:	 40
len lvl:	 22


## Ciclo per il punchout:
Fissata la potenza e la frequenza di readout dal VNA, viene effettuato uno scanning in frequenza e in potenza attraverso il segnale di drive mandato con il Signal Generator. Lo scopo è quello di fare una rappresentazione di punchout in cui si possano osservare i primi quattro livelli energetici del qubit.

**INPUT:** 
	
	- Fissati:
		- Frequenza          (VNA)
		- Potenza            (VNA)
		- Span               (VNA)
		- Bandwidth          (VNA)
		- Number of averages (VNA)
		- Number of points   (VNA)
	- Variabili:
		- Potenza            (SG)
		- Frequenza          (SG)

**OUTPUT:**

	- P_j
		- (f_i, ..., f_k)
			- Media su tutti i punti del modulo I-Q
	.
	.
	.
	- P_s
		- (f_i, ..., f_k)
			- Media su tutti i punti del modulo I-Q
	--------------------------------------------------
	x = Set di frequenze      (SG)
	y = Set di potenze        (SG)
	z = Set di medie ottenute (SG)

In [6]:
import numpy as np
import time
import matplotlib.pyplot as plt

MSR = []
counter = 1
for p in lvl_sg:
	sGen.RF_lvl_ampl(p)
	sGen.RF_state(1)
	amps = []
	for f in freqs_sg:
		print(f'\r{int(counter*100/len(freqs_sg)/len(lvl_sg))}%', end='', flush = True)
		counter += 1
		sGen.RF_freq(f)
		time.sleep(1)
		data = vna.get_data()
		amp = np.mean(5*np.log10(data['Q']**2 + data['I']**2))
		amps.append(amp)
	amps = np.array(amps)
	MSR.append(amps)
	sGen.RF_state(0)
MSR = np.array(MSR)


print(lvl_sg.shape)
print(freqs_sg.shape)
print(MSR.shape)


fig = plt.figure(figsize = (10,6))
plt.pcolor(freqs_sg, lvl_sg, MSR)

plt.show()

0%

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe5 in position 7: ordinal not in range(128)

In [24]:
import plotly.express as px

ampls = MSR
power = lvl_sg
f = freqs_sg

print(ampls.shape)
print(power.shape)
print(f.shape)

fig = px.imshow(ampls, aspect="auto", labels=dict(x = 'Frequency (GHz)', y = 'Power (dBm impostati sul VNA, ricordati di atten)', z = 'MSR'), x = f, y = power, origin = 'lower')

fig.update_layout(
    xaxis=dict(tickformat=".1e"), 
    width = 800,
    height = 500
    )

fig.update_traces(hoverongaps=False, hovertemplate='Frequency: %{x}<br>Amplitude: %{y}<br>MSR: %{z}')

(32, 200)
(32,)
(200,)
